# 실습 08: 표 검출 및 데이터 추출

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/leecks1119/document_ai_lecture/blob/master/notebooks/Lab08_표검출.ipynb)

## 🎯 학습 목표
- 이미지에서 표 영역 검출
- 표 구조 분석 (행/열)
- 표 데이터를 DataFrame으로 변환

## ⏱️ 소요 시간: 40분
## 📊 난이도: ⭐⭐⭐⭐☆


In [ ]:
# 한글 폰트 설치 및 설정 (Colab용)
!apt-get install -y fonts-nanum fonts-nanum-coding fonts-nanum-extra
!fc-cache -fv

# matplotlib 한글 폰트 설정
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=10)
plt.rc('font', family=fontprop.get_name())
plt.rc('axes', unicode_minus=False)

print("✅ 한글 폰트 설정 완료!")


In [ ]:
# 환경 설정
!pip install -q opencv-python-headless paddlepaddle paddleocr
!apt-get install -y tesseract-ocr tesseract-ocr-kor

import cv2
import numpy as np
import pandas as pd
from paddleocr import PaddleOCR
import matplotlib.pyplot as plt


In [ ]:
# 표가 포함된 샘플 이미지 생성
from PIL import Image, ImageDraw, ImageFont

# 한글 폰트 로드
try:
    font_title = ImageFont.truetype("C:\\Windows\\Fonts\\malgunbd.ttf", 28)  # 굵은 글씨
    font_cell = ImageFont.truetype("C:\\Windows\\Fonts\\malgun.ttf", 18)
except:
    try:
        font_title = ImageFont.truetype("/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf", 28)
        font_cell = ImageFont.truetype("/usr/share/fonts/truetype/nanum/NanumGothic.ttf", 18)
    except:
        font_title = ImageFont.load_default()
        font_cell = ImageFont.load_default()

img = Image.new('RGB', (800, 400), color='white')
draw = ImageDraw.Draw(img)

# 제목
draw.text((30, 20), "견적서", fill='black', font=font_title)

# 표 그리기
table_data = [
    ["품목", "수량", "단가", "금액"],
    ["노트북", "10", "1,500,000", "15,000,000"],
    ["모니터", "20", "300,000", "6,000,000"],
    ["키보드", "30", "50,000", "1,500,000"],
]

x_start, y_start = 50, 70
cell_width = 180
cell_height = 40

# 셀 그리기
for i, row in enumerate(table_data):
    for j, cell in enumerate(row):
        x = x_start + j * cell_width
        y = y_start + i * cell_height
        # 테두리
        draw.rectangle([x, y, x+cell_width, y+cell_height], outline='black', width=2)
        # 텍스트
        draw.text((x+10, y+10), cell, fill='black', font=font_cell)

img.save('table_doc.jpg')
print("✅ 표 이미지 생성 완료!")
img


## 💡 표 영역 검출의 중요성

### 문제: 제목과 표 데이터가 섞임
표 외부의 텍스트(제목, 날짜 등)가 표 데이터로 포함되어 잘못된 결과 생성

### 해결: 표 영역만 정확히 검출
```python
# 1. 표 구조 검출 (수평선 + 수직선)
table_structure = cv2.add(horizontal_lines, vertical_lines)

# 2. Contours로 표 경계 찾기
contours = cv2.findContours(table_structure, ...)
table_bbox = cv2.boundingRect(largest_contour)

# 3. 표 영역 내부의 텍스트만 필터링
if (table_x <= text_x <= table_x + table_w and 
    table_y <= text_y <= table_y + table_h):
    # 표 안의 텍스트만 추가
```

### 결과
✅ 표 외부 텍스트 제외 → 정확한 표 데이터 추출


In [ ]:
# OpenCV로 표 영역 검출
img = cv2.imread('table_doc.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

print("="*70)
print("🔧 표 영역 검출 시작")
print("="*70)
print(f"이미지 크기: {img.shape[1]} x {img.shape[0]} (W x H)")

# 이진화
_, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)

# 수평선 검출
horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1))
horizontal_lines = cv2.morphologyEx(binary, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)

# 수직선 검출
vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 40))
vertical_lines = cv2.morphologyEx(binary, cv2.MORPH_OPEN, vertical_kernel, iterations=2)

# 표 구조 결합
table_structure = cv2.add(horizontal_lines, vertical_lines)

# 🔍 표 경계 검출 (Contours)
contours, _ = cv2.findContours(table_structure, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

print(f"\n발견된 Contour 개수: {len(contours)}")
for i, cnt in enumerate(contours, 1):
    area = cv2.contourArea(cnt)
    x, y, w, h = cv2.boundingRect(cnt)
    print(f"  Contour {i}: 면적={area:.0f}, x={x}, y={y}, w={w}, h={h}")

# 가장 큰 contour가 표 영역
table_contour = max(contours, key=cv2.contourArea)
x, y, w, h = cv2.boundingRect(table_contour)

# ⚠️ 표 영역 확장 (경계에 있는 텍스트 포함하되, 제목은 제외)
x_margin = 15  # 좌우 확장 (품목, 키보드 등 왼쪽 텍스트 포함)
y_margin_top = 5  # 위쪽은 최소 확장 (견적서 제목 제외)
y_margin_bottom = 15  # 아래쪽 확장

x = max(0, x - x_margin)
y = max(0, y - y_margin_top)
w = min(img.shape[1] - x, w + 2 * x_margin)
h = min(img.shape[0] - y, h + y_margin_top + y_margin_bottom)

# 표 영역 저장 (나중에 OCR 시 사용)
table_bbox = (x, y, w, h)

print(f"\n✅ 최종 표 영역 (좌우 {x_margin}px, 위 {y_margin_top}px, 아래 {y_margin_bottom}px 확장):")
print(f"  x={x}, y={y}, width={w}, height={h}")
print(f"  범위: X({x}~{x+w}), Y({y}~{y+h})")

# 시각화
img_with_bbox = img.copy()
cv2.rectangle(img_with_bbox, (x, y), (x+w, y+h), (0, 255, 0), 3)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

axes[0, 0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('원본 이미지', fontproperties=fontprop, fontsize=14)
axes[0, 0].axis('off')

axes[0, 1].imshow(cv2.cvtColor(img_with_bbox, cv2.COLOR_BGR2RGB))
axes[0, 1].set_title(f'표 영역 검출 (초록색 박스, 확장됨)', fontproperties=fontprop, fontsize=14)
axes[0, 1].axis('off')

axes[1, 0].imshow(horizontal_lines, cmap='gray')
axes[1, 0].set_title('수평선 검출', fontproperties=fontprop, fontsize=14)
axes[1, 0].axis('off')

axes[1, 1].imshow(vertical_lines, cmap='gray')
axes[1, 1].set_title('수직선 검출', fontproperties=fontprop, fontsize=14)
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# PaddleOCR로 텍스트 추출 및 디버깅
ocr = PaddleOCR(use_textline_orientation=True, lang='korean')
result = ocr.predict('table_doc.jpg')

# OCRResult 객체에서 데이터 추출
ocr_result = result[0]
texts = ocr_result['rec_texts']
scores = ocr_result['rec_scores']
boxes = ocr_result['dt_polys']

print("\n" + "="*80)
print("🔍 디버깅: 전체 OCR 결과 확인")
print("="*80)
print(f"총 {len(texts)}개의 텍스트 영역 검출됨\n")

# 전체 OCR 결과 출력
for i, (text, score, bbox) in enumerate(zip(texts, scores, boxes), 1):
    x_center = (bbox[0][0] + bbox[2][0]) / 2
    y_center = (bbox[0][1] + bbox[2][1]) / 2
    print(f"{i:2d}. '{text:15s}' (신뢰도: {score:.2%}, x={x_center:6.1f}, y={y_center:6.1f})")

# 표 영역 좌표 (이전 셀에서 검출한 table_bbox 사용)
table_x, table_y, table_w, table_h = table_bbox

print(f"\n📍 표 영역 범위:")
print(f"  X: {table_x} ~ {table_x + table_w}")
print(f"  Y: {table_y} ~ {table_y + table_h}")

# ⚠️ 핵심: 표 영역 내부의 텍스트만 필터링
ocr_results = []
filtered_out = []

for text, score, bbox in zip(texts, scores, boxes):
    # 중심점 계산
    x_center = (bbox[0][0] + bbox[2][0]) / 2
    y_center = (bbox[0][1] + bbox[2][1]) / 2
    
    # 🔍 표 영역 내부에 있는 텍스트만 추가
    if (table_x <= x_center <= table_x + table_w and 
        table_y <= y_center <= table_y + table_h):
        ocr_results.append({
            'text': text,
            'x': x_center,
            'y': y_center,
            'confidence': score
        })
    else:
        filtered_out.append({
            'text': text,
            'x': x_center,
            'y': y_center,
            'reason': f"x={x_center:.1f} (범위: {table_x}~{table_x+table_w}), y={y_center:.1f} (범위: {table_y}~{table_y+table_h})"
        })

print(f"\n⚠️ 필터링된 텍스트 ({len(filtered_out)}개):")
for item in filtered_out:
    print(f"  ❌ '{item['text']}' - {item['reason']}")

# Y 좌표로 먼저 정렬 (행), 그 다음 X 좌표로 정렬 (열)
ocr_results_sorted = sorted(ocr_results, key=lambda r: (r['y'], r['x']))

print("\n" + "="*80)
print(f"📋 표 영역 내 텍스트 ({len(ocr_results_sorted)}개, 순서대로):")
print("="*80)
for i, item in enumerate(ocr_results_sorted, 1):
    print(f"{i:2d}. {item['text']:20s} (신뢰도: {item['confidence']:.2%}, x={item['x']:.0f}, y={item['y']:.0f})")
print("="*80)

# 💡 "품목" 누락 확인
if not any(item['text'] == '품목' for item in ocr_results_sorted):
    print("\n❗ 경고: '품목' 텍스트가 표 영역 내에서 발견되지 않았습니다!")
    print("   가능한 원인:")
    print("   1. OCR이 '품목'을 인식하지 못함")
    print("   2. '품목'이 표 영역 바운딩 박스 밖에 위치")
    print("   3. '품목'이 다른 텍스트로 잘못 인식됨")
    
    # "품목"이 전체 OCR 결과에 있는지 확인
    if any(text == '품목' for text in texts):
        print("\n   → '품목'은 OCR로 인식되었으나 표 영역 필터에서 제외됨")
        print("   → 표 영역 바운딩 박스를 확장해야 할 수 있습니다")
    else:
        print("\n   → '품목'이 OCR에서 인식되지 않았습니다")
        print("   → 유사한 텍스트:", [t for t in texts if '품' in t or '목' in t])


In [ ]:
# DataFrame으로 변환 (자동 행/열 구조 분석)
import numpy as np

# 🔍 Y 좌표 분석 (셀 높이 자동 계산)
y_coords = [item['y'] for item in ocr_results_sorted]
print(f"\n📊 Y 좌표 분포 분석:")
print(f"  Y 좌표: {[f'{y:.0f}' for y in y_coords]}")

# Y 좌표 차이 계산 (연속된 텍스트 간)
y_diffs = [y_coords[i+1] - y_coords[i] for i in range(len(y_coords)-1)]
if y_diffs:
    avg_y_diff = np.mean([d for d in y_diffs if d > 0])
    print(f"  평균 Y 좌표 차이: {avg_y_diff:.1f}픽셀")
    # 셀 높이의 절반 정도를 임계값으로 사용
    y_threshold = max(25, avg_y_diff / 2)
else:
    y_threshold = 25

print(f"  자동 계산된 임계값: {y_threshold:.1f}픽셀")

# Y 좌표 기반으로 행 그룹화 (같은 행은 Y 좌표가 비슷함)
rows_grouped = []
current_row = []
current_y = None

for item in ocr_results_sorted:
    if current_y is None:
        current_y = item['y']
        current_row = [item]
    elif abs(item['y'] - current_y) < y_threshold:
        # 같은 행
        current_row.append(item)
    else:
        # 새로운 행
        rows_grouped.append(current_row)
        current_row = [item]
        current_y = item['y']

# 마지막 행 추가
if current_row:
    rows_grouped.append(current_row)

print(f"\n🔍 행별 그룹화 결과 (총 {len(rows_grouped)}개 행):")
for i, row in enumerate(rows_grouped, 1):
    row_texts = [item['text'] for item in row]
    row_y = row[0]['y']
    print(f"  행 {i} (Y={row_y:.0f}): {row_texts}")

# DataFrame 생성
if len(rows_grouped) >= 2:
    # 각 행의 텍스트만 추출
    table_rows = []
    for row in rows_grouped:
        # X 좌표로 정렬 (왼쪽 → 오른쪽)
        sorted_row = sorted(row, key=lambda item: item['x'])
        row_texts = [item['text'] for item in sorted_row]
        table_rows.append(row_texts)
    
    # 모든 행의 열 개수가 같은지 확인
    col_counts = [len(row) for row in table_rows]
    print(f"\n📊 각 행의 열 개수: {col_counts}")
    
    if len(set(col_counts)) == 1:
        # 열 개수가 모두 같음
        df = pd.DataFrame(table_rows[1:], columns=table_rows[0])
        
        print("\n📊 표 데이터 (DataFrame):")
        print("="*80)
        print(df.to_string(index=False))
        print("="*80)
        
        # CSV로 저장
        df.to_csv('extracted_table.csv', index=False, encoding='utf-8-sig')
        print("\n✅ CSV 저장 완료: extracted_table.csv")
        print(f"   - 행 수: {len(df)}개")
        print(f"   - 열 수: {len(df.columns)}개")
    else:
        print("\n⚠️ 경고: 각 행의 열 개수가 다릅니다!")
        print("   행별 데이터:")
        for i, row in enumerate(table_rows, 1):
            print(f"     행 {i}: {row}")
else:
    print("\n⚠️ 표 데이터가 충분하지 않습니다.")
